### Preprocessing

In [35]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
print(train.columns)

Index(['filename', 'width', 'height', 'label', 'xmin', 'ymin', 'xmax', 'ymax'], dtype='object')


In [46]:
import os
def read_directory(directory_name,target):
    array_of_img=[]
    # this loop is for read each image in this foder,directory_name is the foder name with images.
    for i in range(len(target['filename'])):
        img = cv2.imread(directory_name + "/" + target['filename'][i])
        #print(img.shape)
        #print(target['filename'][i])
        IMAGE = img[target['ymin'][i]:target['ymax'][i],target['xmin'][i]:target['xmax'][i],:]
        #print(IMAGE.shape)
        array_of_img.append(IMAGE)
    return array_of_img

In [37]:
def display_one(a, title1 = "Original"):
    plt.imshow(a), plt.title(title1)
    plt.xticks([]), plt.yticks([])
    plt.show()
    #plt.savefig("test.jpg")
def display(a, b, title1 = "Original", title2 = "Edited"):
    plt.subplot(121), plt.imshow(a), plt.title(title1)
    plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(b), plt.title(title2)
    plt.xticks([]), plt.yticks([])
    plt.show()
def processing(data):
    # loading image
    # Getting 3 images to work with
    dim =(64,64)
    res_img=[]
    for img in data:
        res = cv2.resize(img, dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)

    # Visualizing one of the images in the array
    #original = res_img[1]
    #display_one(original)
    return res_img

In [38]:
def denoise(res_img):
    no_noise = []
    for img in (res_img):
        blur = cv2.GaussianBlur(img, (5, 5), 0)
        no_noise.append(blur)
    image = no_noise[1]
    #display(original, image, 'Original', 'Blured')
    return no_noise

In [39]:
bad=[i for i,x in enumerate(train.label) if x=='bad']
good=[i for i,x in enumerate(train.label) if x=='good']
none=[i for i,x in enumerate(train.label) if x=='none']
a=sum(train.label=='bad')
b=sum(train.label=='good')
c=sum(train.label=='none')
print("Good :{},Bad :{},None :{}".format(b,a,c))

Good :2846,Bad :578,None :104


In [40]:
def noisy(low_data):
    mean = 0
    var = 1
    sigma = var**0.5
    noise=[]
    for img in low_data:
        row,col,ch= img.shape
        gauss = np.random.normal(loc=0,scale=1,size = (row,col,ch))
        img=img/255.0
        gaussian_img = np.clip((img*(1+gauss*0.2)),0,1)
        gaussian_img = gaussian_img*255
        noise.append(gaussian_img)
    return noise

In [41]:
def rotate(data):
    rotate_left=[]
    rotate_right=[]
    for img in data:
        h, w = img.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, -90, 1)
        N = cv2.getRotationMatrix2D(center, 90, 1)
        rotated_img = cv2.warpAffine(img, M, (w, h))
        rotated_img_2 = cv2.warpAffine(img, M, (w, h))
        rotate_left.append(rotated_img_2)
        rotate_right.append(rotated_img)
    return rotate_left,rotate_right

In [42]:
labels=np.zeros((len(train),3))
for i,x in enumerate(train.label):
    if x =='good':
        labels[i][0]=1
    elif x=='bad':
        labels[i][1]=1
    else :
        labels[i][2]=1
bad_labels=np.zeros((len(bad),3))
for i in range (len(bad_labels)):
    bad_labels[i][1]=1
none_labels=np.zeros((len(none),3))
for i in range (len(none_labels)):
    none_labels[i][1]=1

In [49]:
x_ = read_directory('images',train)
x_pro=processing(x_)
x_den=denoise(x_pro)
bad_data=[x_pro[x] for x in bad]
bad_noise=noisy(bad_data)
none_data=[x_pro[x] for x in none]
none_noise=noisy(none_data)
x_bad_noises = np.array(bad_noise,dtype=int)
x_none_noises = np.array(none_noise,dtype=int)
left,right=rotate(bad_data)
left_none,right_none = rotate(none_data)

In [50]:
de_none=np.array([x_den[x]for x in none])
data1=np.array(x_pro)
data2=np.array(x_den)
data3=(x_bad_noises)
data4=np.array(right)
data5=np.array(left)
data6=np.array(right_none)
data7=np.array(left_none)
data8=x_none_noises
data = np.concatenate([data1,data3,data4,data5,de_none,data6,data7,data8])
label= np.concatenate([labels,bad_labels,bad_labels,bad_labels,none_labels,none_labels,none_labels,none_labels])
print(len(data))
print(type(data),type(label))
train_loader=zip(data,label)

5678
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


### Dataloader

In [51]:
import torch
import torch.utils.data as Data
import numpy as np
x=torch.from_numpy(data)
y=torch.from_numpy(label)
#x = torch.tensor(train['datas'].values)
#y = torch.tensor(train['labels'].values)

torch.manual_seed(1)    # reproducible
BATCH_SIZE = 30      # 批训练的数据个数
torch_dataset = Data.TensorDataset(x,y)
train_loader = Data.DataLoader(dataset=torch_dataset,      
    batch_size=BATCH_SIZE,shuffle=True,num_workers=2,)

### Model 

In [52]:
import torch.nn as nn
import torch
import torch.nn.functional as F

In [53]:
class CNN(nn.Module):  
    def __init__(self):  
        super(CNN, self).__init__()  
        self.conv1 = nn.Sequential( # (1,28,28)  
                     nn.Conv2d(3, 16, kernel_size=3,  
                               stride=1, padding=1), # (16,28,28)  
        # 想要con2d卷積出來的圖片尺寸沒有變化, padding=(kernel_size-1)/2  
                     nn.ReLU(),  
                     nn.MaxPool2d(2), # (16,14,14)  
                     nn.Dropout2d(0.2)
                     )  
        self.conv2 = nn.Sequential( # (16,14,14)  
                     nn.Conv2d(16, 32, 5, 1, 2), # (32,14,14)  
                     nn.ReLU(),  
                     nn.MaxPool2d(2), # (32,7,7)
                     nn.Dropout2d(0.1)
                     )  
        self.out = nn.Linear(32*7*7, 3)  
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x, 1)
        x = self.out(x)
        output = F.log_softmax(x, 1)
        return output
def train(model,train_data, optimizer, epoch,cuda=False):
    model.train()
    for batch_idx, (data, target) in enumerate(train_data):
        if cuda==True:
            data, target = data.to(cuda), target.to(cuda)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss(output, target)
        loss.backward()
        optimizer.step() 
        if batch_idx % 100 ==0:
            print('Epoch:', epoch, '|Step:', step,  
                  '|train loss:%.4f'%loss.data[0])
def test(model,test_data,cuda=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_data:
            if cuda ==True:
                data, target = data.to(cuda), target.to(cuda)
            output = model(data)
            test_loss += nn.CrossEntropyLoss(output, target)  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_data)


In [57]:
#%store -r train_loader
from torch.optim.lr_scheduler import StepLR
use_cuda = torch.cuda.is_available()
print("use_cuda or not:",use_cuda)
Mask = CNN()
optimizer = torch.optim.Adam(Mask.parameters(), lr=0.01)
lambda1 = lambda epoch: epoch // 30
lambda2 = lambda epoch: 0.95 ** epoch
#scheduler = LambdaLR(optimizer, lr_lambda=[lambda1, lambda2])
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
for epoch in range(100): #每次epoch 都計算train和test的loss
    scheduler.step()
    train(Mask,train_loader,optimizer,epoch,use_cuda)
    #test(Mask, est_loader,use_cuda)
torch.save(Mask.state_dict(), "mask_cnn.pkl")

use_cuda or not: False


OSError: [Errno 12] Cannot allocate memory